# OpenAI Function Calling In LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [ ]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [6]:
foo.age

32

In [ ]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [ ]:
foo_p.name

**Note**: The next cell is expected to fail.

In [ ]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

In [ ]:
class Class(BaseModel):
    students: List[pUser]

In [ ]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [ ]:
obj

## Pydantic to OpenAI function definition


In [8]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [10]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [11]:
weather_function = convert_to_openai_function(WeatherSearch)

In [12]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

In [13]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

**Note**: The next cell is expected to generate an error.

In [15]:
convert_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': "Usage docs: https://docs.pydantic.dev/2.7/concepts/models/ A base class for creating Pydantic models. Attributes:\n    __class_vars__: The names of classvars defined on the model.\n    __private_attributes__: Metadata about the private attributes of the model.\n    __signature__: The signature for instantiating the model.     __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.\n    __pydantic_core_schema__: The pydantic-core schema used to build the SchemaValidator and SchemaSerializer.\n    __pydantic_custom_init__: Whether the model has a custom `__init__` function.\n    __pydantic_decorators__: Metadata containing the decorators defined on the model.\n        This replaces `Model.__validators__` and `Model.__root_validators__` from Pydantic V1.\n    __pydantic_generic_metadata__: Metadata for generic models; contains data used for a similar purpose to\n        __args__, __origin__, __paramete

In [16]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [17]:
convert_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

In [18]:
from langchain_openai import ChatOpenAI

In [19]:
model = ChatOpenAI()

In [20]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-650b1723-0599-4a28-b7a7-eac94c7a76ee-0')

In [21]:
model_with_function = model.bind(functions=[weather_function])

In [22]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9dd3162a-e76b-40d9-80bf-4d46d84a8d99-0')

## Forcing it to use a function

We can force the model to use a function

In [23]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [24]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 70, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dd8daeca-8ff2-48b2-99e9-90e16f4be93b-0')

In [25]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 65, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-08495f76-43a1-4caf-9381-fc875c3018c4-0')

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [26]:
from langchain.prompts import ChatPromptTemplate

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [28]:
chain = prompt | model_with_function

In [30]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 66, 'total_tokens': 83}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4c5f9b6d-eb38-4b32-8486-327b49c7287d-0')

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [31]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [32]:
functions = [
    convert_to_openai_function(WeatherSearch),
    convert_to_openai_function(ArtistSearch),
]

In [33]:
model_with_functions = model.bind(functions=functions)

In [34]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-078542f9-73a0-4323-85e8-07a2a5d1fb45-0')

In [35]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ac0ab78c-5fbe-4314-9331-f83d1323ca46-0')

In [36]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 88, 'total_tokens': 98}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3aa8dddc-7e52-47b4-9e1e-209982ff0b74-0')